# 1) Tratamento das bases de dados

As bases de dados utilizados são dados advindos do Banco Central do Brasil, mais especificamente do Sistema Gerenciador de Séries temporais e a tabela de Metas de taxa para a Selic

## Coletando as séries do Sistema Gerenciador de Séries Temporais / BCB

In [25]:
import time
import math
import requests  # cliente HTTP
import pandas as pd
from datetime import datetime
from pathlib import Path

URL_BASE = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados"

def baixar_sgs(codigo: int, data_inicial: str, data_final: str, tentativas_max: int = 5) -> pd.DataFrame:
    """
    Baixa UMA série do SGS em JSON e devolve DataFrame com colunas: ['dt', f'sgs_{codigo}'].
    data_inicial/data_final no formato 'dd/mm/aaaa'. Backoff simples em caso de erro HTTP.
    """
    parametros = {"formato": "json", "dataInicial": data_inicial, "dataFinal": data_final}
    for tentativa in range(tentativas_max):
        try:
            resp = requests.get(URL_BASE.format(codigo=codigo), params=parametros, timeout=30)
            resp.raise_for_status()
            dados = resp.json()
            df = pd.DataFrame(dados)
            if df.empty:
                return pd.DataFrame(columns=["dt", f"sgs_{codigo}"])
            # Converte data (pode vir dd/mm/aaaa ou mm/aaaa)
            def parsear_data(s):
                for fmt in ("%d/%m/%Y", "%m/%Y"):
                    try:
                        return datetime.strptime(s, fmt)
                    except ValueError:
                        pass
                return pd.NaT
            df["dt"] = df["data"].apply(parsear_data)
            df[f"sgs_{codigo}"] = pd.to_numeric(df["valor"].str.replace(",", "."), errors="coerce")
            saida = df[["dt", f"sgs_{codigo}"]].dropna(subset=["dt"]).copy()
            # normaliza para frequência mensal (MS)
            saida = (
                saida.set_index("dt")
                     .sort_index()
                     .asfreq("MS")  # Month Start
                     .ffill()       # preenche mês faltante com último valor
            )
            saida.reset_index(inplace=True)
            return saida
        except requests.RequestException as e:
            espera = 2 ** tentativa
            time.sleep(espera)
            if tentativa == tentativas_max - 1:
                raise e

def baixar_varias(codigos: list[int], data_inicial: str, data_final: str) -> pd.DataFrame:
    """
    Faz merge (outer) de várias séries por 'dt'. Retorna DataFrame com dt + colunas sgs_XXXX.
    """
    df_todas = None
    for codigo in codigos:
        df_serie = baixar_sgs(codigo, data_inicial, data_final)
        df_todas = df_serie if df_todas is None else df_todas.merge(df_serie, on="dt", how="outer")
    # index mensal e ordenação
    return df_todas.set_index("dt").sort_index().asfreq("MS")

# conjuntos de séries que serão utilizadas na análise
CODIGOS_LIVRES = [20635, 20662, 20634, 20718, 20740, 20717, 20787, 20809, 20786]
CODIGOS_ICC_LIVRES = [25355, 25356, 25354]  # opcional

CODIGOS_DIRECIONADOS = [20686, 20698, 20685, 20757, 20768, 20756, 20826, 20837, 20825]
CODIGOS_ICC_DIRECIONADOS = [25358, 25359, 25357]  # opcional

INCLUIR_ICC = True
CODIGOS_LIVRES_FINAIS = CODIGOS_LIVRES + (CODIGOS_ICC_LIVRES if INCLUIR_ICC else [])
CODIGOS_DIRECIONADOS_FINAIS = CODIGOS_DIRECIONADOS + (CODIGOS_ICC_DIRECIONADOS if INCLUIR_ICC else [])

# datas-alvo
DATA_INICIAL = "01/01/2016"
DATA_FINAL = datetime.today().strftime("%d/%m/%Y")

# baixa e organiza 
df_livres = baixar_varias(CODIGOS_LIVRES_FINAIS, DATA_INICIAL, DATA_FINAL)
df_direcionados = baixar_varias(CODIGOS_DIRECIONADOS_FINAIS, DATA_INICIAL, DATA_FINAL)


conjunto_dados = df_livres.join(df_direcionados, how="outer")

conjunto_dados


,sgs_20635,sgs_20662,sgs_20634,sgs_20718,sgs_20740,sgs_20717,sgs_20787,sgs_20809,sgs_20786,sgs_25355,...,sgs_20685,sgs_20757,sgs_20768,sgs_20756,sgs_20826,sgs_20837,sgs_20825,sgs_25358,sgs_25359,sgs_25357
dt,,,,,,,,,,,,,,,,,,,,,
2016-01-01,97151,103508,200658,31.34,65.80,48.78,16.81,50.02,33.62,25.21,...,20433,11.62,9.94,10.71,4.29,3.65,3.94,8.41,8.48,8.44
2016-02-01,99433,106054,205487,31.72,67.33,49.88,17.51,52.10,35.15,25.57,...,20736,11.78,9.69,10.64,4.48,3.56,3.98,8.44,8.51,8.47
2016-03-01,118772,112509,231281,30.66,68.57,50.07,17.60,54.39,36.44,25.66,...,29000,11.89,10.13,10.93,4.56,3.76,4.12,8.52,8.55,8.53
2016-04-01,105583,106139,211721,30.68,70.42,51.21,17.81,56.98,38.05,25.78,...,27932,11.54,10.03,10.71,4.21,3.77,3.97,8.56,8.58,8.57
2016-05-01,112566,114529,227095,29.89,70.70,51.12,17.69,57.69,38.50,25.61,...,27227,11.81,10.46,11.07,4.43,4.00,4.20,8.59,8.63,8.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,259516,301404,560920,23.47,56.77,43.86,9.75,42.30,29.68,20.60,...,59003,18.32,11.37,12.92,5.09,4.47,4.61,10.97,9.54,10.01
2025-04-01,263452,309881,573332,24.21,58.01,45.01,10.65,43.75,31.02,20.88,...,70925,15.85,11.14,12.21,4.35,4.47,4.44,11.14,9.57,10.09
2025-05-01,271402,307525,578928,24.17,58.70,45.64,10.76,44.71,31.87,20.93,...,68547,14.67,11.27,12.04,4.50,4.46,4.47,11.21,9.60,10.14


## Transposição da base: As séries serão linhas e as colunas as datas (mês/ano)

In [26]:
# 1) Garantir ordenação por data e formato amigável do rótulo de coluna (datas)
conjunto_dados = conjunto_dados.sort_index()
rotulos_datas = [dt.strftime("%Y-%m") for dt in conjunto_dados.index]

# 2) Transpor: linhas = séries, colunas = datas
tabela_series = conjunto_dados.T.copy()
tabela_series.columns = rotulos_datas

# 3) (Opcional) Renomear o índice (nome das linhas) para algo mais intuitivo
tabela_series.index.name = "Série"

tabela_series

,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
Série,,,,,,,,,,,,,,,,,,,,,
sgs_20635,97151.00,99433.00,118772.00,105583.00,112566.00,115862.00,95447.00,102841.00,110601.00,101911.00,...,257922.00,253835.00,310403.00,238004.00,237764.00,259516.00,263452.00,271402.00,264523.00,253797.00
sgs_20662,103508.00,106054.00,112509.00,106139.00,114529.00,113802.00,110430.00,119029.00,108839.00,114147.00,...,309554.00,302438.00,316382.00,302714.00,292589.00,301404.00,309881.00,307525.00,304322.00,317637.00
sgs_20634,200658.00,205487.00,231281.00,211721.00,227095.00,229664.00,205877.00,221869.00,219440.00,216059.00,...,567476.00,556273.00,626785.00,540718.00,530276.00,560920.00,573332.00,578928.00,568844.00,571434.00
sgs_20718,31.34,31.72,30.66,30.68,29.89,30.04,30.32,30.50,29.51,30.24,...,21.38,21.66,21.73,24.13,23.78,23.47,24.21,24.17,24.29,25.02
sgs_20740,65.80,67.33,68.57,70.42,70.70,70.18,70.75,71.15,72.61,72.87,...,52.39,53.16,53.11,54.28,56.64,56.77,58.01,58.70,58.42,57.65
sgs_20717,48.78,49.88,50.07,51.21,51.12,50.93,51.60,52.10,52.43,53.06,...,40.17,40.86,40.74,42.46,43.85,43.86,45.01,45.64,45.51,45.40
sgs_20787,16.81,17.51,17.60,17.81,17.69,17.86,18.29,18.40,17.65,18.55,...,9.82,9.62,8.56,10.61,10.22,9.75,10.65,10.76,10.91,11.61
sgs_20809,50.02,52.10,54.39,56.98,57.69,57.25,58.05,58.51,60.09,60.82,...,40.25,40.54,39.25,39.90,42.35,42.30,43.75,44.71,44.40,43.64
sgs_20786,33.62,35.15,36.44,38.05,38.50,38.36,39.21,39.71,40.22,41.18,...,28.26,28.46,27.16,28.42,29.84,29.68,31.02,31.87,31.73,31.61


### Renomeando as linhas para tornar os códigos identificáveis com seus respectivos indicadores

In [27]:
mapa_series = {
    # --------- RECURSOS LIVRES (RL) ---------
    "sgs_20635": "concessoes_RL_PJ_20635",
    "sgs_20662": "concessoes_RL_PF_20662",
    "sgs_20634": "concessoes_RL_TOT_20634",

    "sgs_20718": "taxa_RL_PJ_20718",
    "sgs_20740": "taxa_RL_PF_20740",
    "sgs_20717": "taxa_RL_TOT_20717",

    "sgs_20787": "spread_RL_PJ_20787",
    "sgs_20809": "spread_RL_PF_20809",
    "sgs_20786": "spread_RL_TOT_20786",

    "sgs_25355": "icc_RL_PJ_25355",
    "sgs_25356": "icc_RL_PF_25356",
    "sgs_25354": "icc_RL_TOT_25354",

    # --------- RECURSOS DIRECIONADOS (RD) ---------
    "sgs_20686": "concessoes_RD_PJ_20686",
    "sgs_20698": "concessoes_RD_PF_20698",
    "sgs_20685": "concessoes_RD_TOT_20685",

    "sgs_20757": "taxa_RD_PJ_20757",
    "sgs_20768": "taxa_RD_PF_20768",
    "sgs_20756": "taxa_RD_TOT_20756",

    "sgs_20826": "spread_RD_PJ_20826",
    "sgs_20837": "spread_RD_PF_20837",
    "sgs_20825": "spread_RD_TOT_20825",

    "sgs_25358": "icc_RD_PJ_25358",
    "sgs_25359": "icc_RD_PF_25359",
    "sgs_25357": "icc_RD_TOT_25357",
}

# renomeia o índice (linhas) usando o dicionário
tabela_series = tabela_series.rename(index=mapa_series)

tabela_series

,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
Série,,,,,,,,,,,,,,,,,,,,,
concessoes_RL_PJ_20635,97151.00,99433.00,118772.00,105583.00,112566.00,115862.00,95447.00,102841.00,110601.00,101911.00,...,257922.00,253835.00,310403.00,238004.00,237764.00,259516.00,263452.00,271402.00,264523.00,253797.00
concessoes_RL_PF_20662,103508.00,106054.00,112509.00,106139.00,114529.00,113802.00,110430.00,119029.00,108839.00,114147.00,...,309554.00,302438.00,316382.00,302714.00,292589.00,301404.00,309881.00,307525.00,304322.00,317637.00
concessoes_RL_TOT_20634,200658.00,205487.00,231281.00,211721.00,227095.00,229664.00,205877.00,221869.00,219440.00,216059.00,...,567476.00,556273.00,626785.00,540718.00,530276.00,560920.00,573332.00,578928.00,568844.00,571434.00
taxa_RL_PJ_20718,31.34,31.72,30.66,30.68,29.89,30.04,30.32,30.50,29.51,30.24,...,21.38,21.66,21.73,24.13,23.78,23.47,24.21,24.17,24.29,25.02
taxa_RL_PF_20740,65.80,67.33,68.57,70.42,70.70,70.18,70.75,71.15,72.61,72.87,...,52.39,53.16,53.11,54.28,56.64,56.77,58.01,58.70,58.42,57.65
taxa_RL_TOT_20717,48.78,49.88,50.07,51.21,51.12,50.93,51.60,52.10,52.43,53.06,...,40.17,40.86,40.74,42.46,43.85,43.86,45.01,45.64,45.51,45.40
spread_RL_PJ_20787,16.81,17.51,17.60,17.81,17.69,17.86,18.29,18.40,17.65,18.55,...,9.82,9.62,8.56,10.61,10.22,9.75,10.65,10.76,10.91,11.61
spread_RL_PF_20809,50.02,52.10,54.39,56.98,57.69,57.25,58.05,58.51,60.09,60.82,...,40.25,40.54,39.25,39.90,42.35,42.30,43.75,44.71,44.40,43.64
spread_RL_TOT_20786,33.62,35.15,36.44,38.05,38.50,38.36,39.21,39.71,40.22,41.18,...,28.26,28.46,27.16,28.42,29.84,29.68,31.02,31.87,31.73,31.61


## Importando o CSV da Taxa Selic

In [28]:
# 1) Ler CSV da Selic e obter a meta vigente mensal (% a.a.)
caminho_csv = Path("meta-para-a-taxa-selic.csv")

# ler CSV
try:
    df_selic = pd.read_csv(caminho_csv, sep=';', encoding='utf-8-sig')
except UnicodeDecodeError:
    df_selic = pd.read_csv(caminho_csv, sep=';', encoding='latin-1')

# renomear
df_selic = df_selic.rename(columns={
    "DateTime": "data_evento",
    "Meta para a taxa Selic": "selic_meta_aa"
})

# parse ISO, filtra nulos
df_selic["data_evento"] = pd.to_datetime(df_selic["data_evento"], format="%Y-%m-%d", errors="coerce")
df_selic = df_selic.dropna(subset=["data_evento"])

# reduzir p/ ano-mês e ancorar no 1º dia do mês
df_selic["data_evento"] = df_selic["data_evento"].dt.to_period("M").dt.to_timestamp()

# ordena e remove duplicatas mensais (fica com o último registro do mês)
df_selic = (df_selic.sort_values("data_evento")
                     .drop_duplicates(subset=["data_evento"], keep="last")
                     .reset_index(drop=True))

# normalizando igual fizemos com a tabela do sgs
df_selic["selic_meta_aa"] = pd.to_numeric(
    df_selic["selic_meta_aa"].astype(str).str.replace(",", "."),
    errors="coerce"
)

df_selic["ano_mes"] = df_selic["data_evento"].dt.strftime("%Y-%m")

df_selic

,data_evento,selic_meta_aa,ano_mes
0,2015-09-01,14.25,2015-09
1,2015-10-01,14.25,2015-10
2,2015-11-01,14.25,2015-11
3,2015-12-01,14.25,2015-12
4,2016-01-01,14.25,2016-01
...,...,...,...
116,2025-05-01,14.25,2025-05
117,2025-06-01,14.75,2025-06
118,2025-07-01,15.00,2025-07
119,2025-08-01,15.00,2025-08


## Transpondo a base para concatenar com a "tabela_series"

In [31]:
# colunas ordenadas pelo 'ano_mes' disponível
colunas_anomes = sorted(df_selic["ano_mes"].unique())

# transposição
tabela_selic = (
    df_selic.set_index("ano_mes")["selic_meta_aa"]
            .reindex(colunas_anomes)     # garante ordem das colunas
            .rename("selic_aa")          # nome da LINHA
            .to_frame().T                # vira DataFrame de 1 linha
)

# mantem apenas as colunas de 2013-01 adiante, assim como na tabela_series
colunas_filtradas = [c for c in tabela_selic.columns if c >= "2016-01" and c <= "2025-07"]
tabela_selic = tabela_selic.loc[:, colunas_filtradas]

tabela_selic

ano_mes,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
selic_aa,14.25,14.25,14.25,14.25,14.25,14.25,14.25,14.25,14.25,14.25,...,10.75,11.25,11.25,12.25,13.25,13.25,14.25,14.25,14.75,15.0


## Junção das bases

In [32]:
base = pd.concat([tabela_series, tabela_selic])

base

,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
concessoes_RL_PJ_20635,97151.00,99433.00,118772.00,105583.00,112566.00,115862.00,95447.00,102841.00,110601.00,101911.00,...,257922.00,253835.00,310403.00,238004.00,237764.00,259516.00,263452.00,271402.00,264523.00,253797.00
concessoes_RL_PF_20662,103508.00,106054.00,112509.00,106139.00,114529.00,113802.00,110430.00,119029.00,108839.00,114147.00,...,309554.00,302438.00,316382.00,302714.00,292589.00,301404.00,309881.00,307525.00,304322.00,317637.00
concessoes_RL_TOT_20634,200658.00,205487.00,231281.00,211721.00,227095.00,229664.00,205877.00,221869.00,219440.00,216059.00,...,567476.00,556273.00,626785.00,540718.00,530276.00,560920.00,573332.00,578928.00,568844.00,571434.00
taxa_RL_PJ_20718,31.34,31.72,30.66,30.68,29.89,30.04,30.32,30.50,29.51,30.24,...,21.38,21.66,21.73,24.13,23.78,23.47,24.21,24.17,24.29,25.02
taxa_RL_PF_20740,65.80,67.33,68.57,70.42,70.70,70.18,70.75,71.15,72.61,72.87,...,52.39,53.16,53.11,54.28,56.64,56.77,58.01,58.70,58.42,57.65
taxa_RL_TOT_20717,48.78,49.88,50.07,51.21,51.12,50.93,51.60,52.10,52.43,53.06,...,40.17,40.86,40.74,42.46,43.85,43.86,45.01,45.64,45.51,45.40
spread_RL_PJ_20787,16.81,17.51,17.60,17.81,17.69,17.86,18.29,18.40,17.65,18.55,...,9.82,9.62,8.56,10.61,10.22,9.75,10.65,10.76,10.91,11.61
spread_RL_PF_20809,50.02,52.10,54.39,56.98,57.69,57.25,58.05,58.51,60.09,60.82,...,40.25,40.54,39.25,39.90,42.35,42.30,43.75,44.71,44.40,43.64
spread_RL_TOT_20786,33.62,35.15,36.44,38.05,38.50,38.36,39.21,39.71,40.22,41.18,...,28.26,28.46,27.16,28.42,29.84,29.68,31.02,31.87,31.73,31.61
icc_RL_PJ_25355,25.21,25.57,25.66,25.78,25.61,25.83,25.96,25.97,26.07,26.24,...,19.45,19.48,19.35,20.13,20.38,20.60,20.88,20.93,21.34,21.38
